In [3]:
import pandas as pd
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt

from shapely.wkt import loads as wkt_loads

in this profile, we got nut3_2013 from NUTS folder saved in data, and we loaded the hotmaps_industrial database
then we changed both of them to geodataframe, and used code 4326
then mapped both files on geometry, so that each industry has its nuts_id now
at the end i combined nuts3 to nuts2, and now i have number of indutries sectorwise in each 4 clusters, 

In [4]:


# Path to your CSV file
file_path = "C:/Users/86435/project_h2/data/heat_demand/industry/for_weights/Industrial_Database_hotmaps.csv"

# Load using semicolon (;) as delimiter
df_industrial_db = pd.read_csv(file_path, delimiter=';')

df = df_industrial_db[df_industrial_db['Country'] == 'Germany'].copy()

# Step 2: Filter out rows with invalid or missing geom values
df_valid = df[df['geom'].apply(lambda x: isinstance(x, str))].copy()

# Step 3: Convert WKT geom to shapely geometry
df_valid['geometry'] = df_valid['geom'].str.replace('SRID=4326;', '', regex=False).apply(wkt_loads)

# Step 4: Create GeoDataFrame

gdf_industry = gpd.GeoDataFrame(df_valid, geometry='geometry', crs='EPSG:4326')  # starts as lat/lon



In [5]:
# setting up nuts for NUTS3_2013 file
file_path1 = "C:/Users/86435/project_h2/data/NUTS/NUTS3_2013.csv"
nuts = pd.read_csv(file_path1)

df_nuts = nuts[nuts['NUTS_ID'].str.startswith('DE')].copy()
# Load NUTS polygons
df_nuts['geometry'] = df_nuts['WKT'].apply(wkt_loads)
gdf_nuts = gpd.GeoDataFrame(df_nuts, geometry='geometry', crs='EPSG:3035')
gdf_nuts['geometry'] = gdf_nuts['geometry'].buffer(0)  # Fix invalid geometries
gdf_nuts = gdf_nuts.to_crs(epsg=4326)



In [6]:
point = gdf_industry.geometry.iloc[0]

for i, row in gdf_nuts.iterrows():
    if row['geometry'].contains(point):
        print(f"Point is in NUTS region: {row['NUTS_ID']}")
        break
else:
    print("No matching polygon found for this point.")


Point is in NUTS region: DE11D


In [7]:
# Perform spatial join to assign NUTS_ID
gdf_joined = gpd.sjoin(
    gdf_industry,
    gdf_nuts[['NUTS_ID', 'geometry']],
    how='left',
    predicate='within'
)

prep for 4 clusters


In [34]:
# Keep only the relevant columns
df_cleaned = gdf_joined[['CompanyName', 'Subsector', 'Emissions_ETS_2014', 'Production', 'Fuel_Demand', 'NUTS_ID']].copy()

# Set NUTS_ID as the index
df_cleaned.set_index('NUTS_ID', inplace=True)

In [35]:
df_cleaned.columns

Index(['CompanyName', 'Subsector', 'Emissions_ETS_2014', 'Production',
       'Fuel_Demand'],
      dtype='object')

In [36]:
# Cluster mapping dictionary
CLUSTER_MAP = {
    'DE9': ['DE5', 'DE6', 'DE8', 'DE9', 'DEF'],
    'DEA': ['DE7', 'DEA', 'DEB', 'DEC'],
    'DE2': ['DE1', 'DE2'],
    'DEE': ['DE3', 'DE4', 'DED', 'DEE', 'DEG'],
}

# Reverse map: prefix -> cluster
prefix_to_cluster = {prefix: cluster for cluster, prefixes in CLUSTER_MAP.items() for prefix in prefixes}

# Function to match NUTS_ID to cluster
def assign_cluster(nuts_id):
    if pd.isna(nuts_id):
        return None
    for prefix in prefix_to_cluster:
        if nuts_id.startswith(prefix):
            return prefix_to_cluster[prefix]
    return 'Other'

# Apply to DataFrame
df_cleaned['Cluster'] = df_cleaned.index.to_series().apply(assign_cluster)


In [37]:

# Preview result
df_cleaned['Cluster'].value_counts()

DEA    332
DE2    180
DEE    140
DE9    108
Name: Cluster, dtype: int64

In [44]:
df_cleaned.columns

Index(['CompanyName', 'Subsector', 'Emissions_ETS_2014', 'Production',
       'Fuel_Demand', 'Cluster'],
      dtype='object')